# Does resampling experiment help with predicting CtD sentences?

In [1]:
from itertools import product
from pathlib import Path
import warnings

import pandas as pd
import plydata as ply
from sqlalchemy import create_engine

from snorkel.labeling.analysis import LFAnalysis
from snorkeling_helper.generative_model_helper import (
    sample_lfs,
    train_generative_label_function_sampler,
)

warnings.filterwarnings("ignore")

In [2]:
username = "danich1"
password = "snorkel"
dbname = "pubmed_central_db"
database_str = (
    f"postgresql+psycopg2://{username}:{password}@/{dbname}?host=/var/run/postgresql"
)
conn = create_engine(database_str)

## Load the data

In [3]:
label_candidates_dir = Path("../label_candidates/output")
notebook_output_dir = Path("output/CtD")

In [3]:
L_abstracts = pd.read_csv(
    str(label_candidates_dir / Path("cd_abstract_train_candidates_resampling.tsv")),
    sep="\t",
)

print(L_abstracts.shape)
L_abstracts.head().T

(971820, 103)


,0,1,2,3,4
LF_HETNET_PHARMACOTHERAPYDB,-1,-1,-1,-1,-1
LF_HETNET_CD_ABSENT,-1,-1,-1,-1,-1
LF_CD_CHECK_DISEASE_TAG,0,0,0,0,-1
LF_DG_IS_BIOMARKER,-1,-1,-1,-1,-1
LF_DaG_ASSOCIATION,-1,-1,-1,-1,-1
...,...,...,...,...,...
LF_GG_BICLUSTER_INCREASES_EXPRESSION,-1,-1,-1,-1,-1
LF_GG_BICLUSTER_SIGNALING,-1,-1,-1,-1,-1
LF_GG_BICLUSTER_IDENTICAL_PROTEIN,-1,-1,-1,-1,-1
LF_GG_BICLUSTER_CELL_PRODUCTION,-1,0,-1,-1,-1


In [4]:
L_full_text = pd.read_csv(
    str(label_candidates_dir / Path("cd_full_text_train_candidates_resampling.tsv")),
    sep="\t",
)

print(L_full_text.shape)
L_full_text.head().T

(1247188, 103)


,0,1,2,3,4
LF_HETNET_PHARMACOTHERAPYDB,-1,-1,-1,-1,-1
LF_HETNET_CD_ABSENT,-1,-1,-1,-1,-1
LF_CD_CHECK_DISEASE_TAG,0,0,0,0,0
LF_DG_IS_BIOMARKER,-1,-1,-1,-1,-1
LF_DaG_ASSOCIATION,-1,-1,-1,-1,-1
...,...,...,...,...,...
LF_GG_BICLUSTER_INCREASES_EXPRESSION,-1,-1,-1,-1,-1
LF_GG_BICLUSTER_SIGNALING,-1,-1,-1,-1,-1
LF_GG_BICLUSTER_IDENTICAL_PROTEIN,-1,-1,-1,-1,-1
LF_GG_BICLUSTER_CELL_PRODUCTION,-1,-1,-1,-1,-1


In [5]:
L_dev = pd.read_csv(
    str(label_candidates_dir / Path("cd_dev_test_candidates_resampling.tsv")), sep="\t"
) >> ply.query("split==10")
print(L_dev.shape)
L_dev.head().T

(500, 105)


,1,2,3,4,6
LF_HETNET_PHARMACOTHERAPYDB,-1.0,-1.0,-1.0,1.0,1.0
LF_HETNET_CD_ABSENT,-1.0,-1.0,-1.0,-1.0,-1.0
LF_CD_CHECK_DISEASE_TAG,-1.0,0.0,0.0,-1.0,-1.0
LF_DG_IS_BIOMARKER,-1.0,-1.0,-1.0,-1.0,-1.0
LF_DaG_ASSOCIATION,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...
LF_GG_BICLUSTER_IDENTICAL_PROTEIN,-1.0,-1.0,-1.0,-1.0,-1.0
LF_GG_BICLUSTER_CELL_PRODUCTION,-1.0,-1.0,-1.0,-1.0,-1.0
split,10.0,10.0,10.0,10.0,10.0
document_id,162660.0,146415.0,213795.0,33875.0,15752.0


## Resort the Candidates Based on Abstract

In [6]:
# Grab the document ids for resampling
sql = """
select cd_candidates.sentence_id, document_id, cd_candidates.candidate_id
from sentence
inner join (
  select candidate.candidate_id, compound_disease.sentence_id from compound_disease
  inner join candidate on candidate.candidate_id=compound_disease.candidate_id
  ) as cd_candidates
on sentence.sentence_id = cd_candidates.sentence_id
"""
candidate_doc_df = pd.read_sql(sql, database_str)
candidate_doc_df.head()

,sentence_id,document_id,candidate_id
0,578151345,24670128,8160
1,568187139,27952167,9876
2,561956917,8224816,20087
3,244310297,28588380,45149
4,593065266,15261556,89224


In [7]:
dev_test_ids = (
    L_dev >> ply.select("document_id") >> ply.distinct() >> ply.pull("document_id")
)

filtered_candidate_id = (
    candidate_doc_df
    >> ply.query(f"document_id in {list(dev_test_ids)}")
    >> ply.pull("candidate_id")
)

In [8]:
sorted_train_df = pd.read_csv(
    str(notebook_output_dir / Path("ctd_dataset_mapper.tsv")), sep="\t"
)
sorted_train_df.head()

,document_id,dataset
0,24670128,train
1,27952167,train
2,8224816,train
3,28588380,tune
4,15261556,train


In [9]:
trained_documents = (
    sorted_train_df
    >> ply.inner_join(candidate_doc_df, on="document_id")
    >> ply.query("dataset=='train'")
    >> ply.pull("candidate_id")
)

In [10]:
filtered_L_abstracts = L_abstracts >> ply.query(
    f"candidate_id in {list(trained_documents)}"
)
print(filtered_L_abstracts.shape)
filtered_L_abstracts.head()

(680470, 103)


,LF_HETNET_PHARMACOTHERAPYDB,LF_HETNET_CD_ABSENT,LF_CD_CHECK_DISEASE_TAG,LF_DG_IS_BIOMARKER,LF_DaG_ASSOCIATION,LF_DaG_WEAK_ASSOCIATION,LF_DaG_NO_ASSOCIATION,LF_DaG_CELLULAR_ACTIVITY,LF_DaG_DISEASE_SAMPLE,LF_DG_METHOD_DESC,...,LF_GG_NO_VERB,LF_GG_BICLUSTER_BINDING,LF_GG_BICLUSTER_ENHANCES,LF_GG_BICLUSTER_ACTIVATES,LF_GG_BICLUSTER_AFFECTS_EXPRESSION,LF_GG_BICLUSTER_INCREASES_EXPRESSION,LF_GG_BICLUSTER_SIGNALING,LF_GG_BICLUSTER_IDENTICAL_PROTEIN,LF_GG_BICLUSTER_CELL_PRODUCTION,candidate_id
0,-1,-1,0,-1,-1,-1,-1,-1,1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,52
3,-1,-1,0,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,408
4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,409
5,-1,-1,0,-1,-1,-1,-1,-1,1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,410
6,-1,-1,-1,-1,-1,-1,-1,-1,1,0,...,-1,-1,-1,1,-1,-1,-1,-1,-1,413


In [11]:
filtered_L_full_text = L_full_text >> ply.query(
    f"candidate_id in {list(trained_documents)}"
)
print(filtered_L_full_text.shape)
filtered_L_full_text.head()

(873241, 103)


,LF_HETNET_PHARMACOTHERAPYDB,LF_HETNET_CD_ABSENT,LF_CD_CHECK_DISEASE_TAG,LF_DG_IS_BIOMARKER,LF_DaG_ASSOCIATION,LF_DaG_WEAK_ASSOCIATION,LF_DaG_NO_ASSOCIATION,LF_DaG_CELLULAR_ACTIVITY,LF_DaG_DISEASE_SAMPLE,LF_DG_METHOD_DESC,...,LF_GG_NO_VERB,LF_GG_BICLUSTER_BINDING,LF_GG_BICLUSTER_ENHANCES,LF_GG_BICLUSTER_ACTIVATES,LF_GG_BICLUSTER_AFFECTS_EXPRESSION,LF_GG_BICLUSTER_INCREASES_EXPRESSION,LF_GG_BICLUSTER_SIGNALING,LF_GG_BICLUSTER_IDENTICAL_PROTEIN,LF_GG_BICLUSTER_CELL_PRODUCTION,candidate_id
30,-1,-1,0,-1,1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,233
31,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,234
32,-1,-1,0,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,420
33,-1,-1,0,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,422
34,-1,-1,0,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,424


## Construct the Grid Search

In [12]:
# Global Grid
epochs_grid = [100]
l2_param_grid = [0.75]
lr_grid = [1e-3]
grid = list(product(epochs_grid, l2_param_grid, lr_grid))

# Abstracts

In [13]:
analysis_module = LFAnalysis(
    filtered_L_abstracts >> ply.select("candidate_id", drop=True)
)

abstract_lf_summary = analysis_module.lf_summary()
abstract_lf_summary.index = (
    filtered_L_abstracts >> ply.select("candidate_id", drop=True)
).columns.tolist()

abstract_lf_summary

,Polarity,Coverage,Overlaps,Conflicts
LF_HETNET_PHARMACOTHERAPYDB,[1],0.263360,0.263360,0.263360
LF_HETNET_CD_ABSENT,[],0.000000,0.000000,0.000000
LF_CD_CHECK_DISEASE_TAG,[0],0.736698,0.736698,0.736698
LF_DG_IS_BIOMARKER,[1],0.070054,0.070054,0.070054
LF_DaG_ASSOCIATION,[1],0.091406,0.091406,0.091406
...,...,...,...,...
LF_GG_BICLUSTER_AFFECTS_EXPRESSION,[1],0.014456,0.014456,0.014456
LF_GG_BICLUSTER_INCREASES_EXPRESSION,[0],0.033119,0.033119,0.033119
LF_GG_BICLUSTER_SIGNALING,[1],0.033039,0.033039,0.033039
LF_GG_BICLUSTER_IDENTICAL_PROTEIN,[0],0.019763,0.019763,0.019763


# Set up fields for resampling

In [14]:
lf_columns_base = list(L_abstracts.columns[0:5])
candidate_id_field = list(L_abstracts.columns[-1:])
dev_column_base = ["split", "curated_ctd", "document_id"]
data_columns = []

# Abstracts

## baseline

In [15]:
ctd_start = 0
ctd_end = 2
number_of_samples = 1

ctd_lf_range = range(ctd_start, ctd_end)
size_of_samples = [len(ctd_lf_range)]

In [16]:
sampled_lfs_dict = {
    sample_size: (
        sample_lfs(
            list(ctd_lf_range),
            len(list(ctd_lf_range)),
            sample_size,
            number_of_samples,
            random_state=100,
        )
    )
    for sample_size in size_of_samples
}

In [17]:
data_columns += train_generative_label_function_sampler(
    filtered_L_abstracts,
    L_dev,
    sampled_lfs_dict,
    lf_columns_base=lf_columns_base,
    candidate_id_field=candidate_id_field,
    dev_column_base=dev_column_base,
    search_grid=grid,
    marginals_df_file=str(
        notebook_output_dir / Path("ctd_training_marginals_baseline.tsv")
    ),
    curated_label="curated_ctd",
    entity_label="CtD",
    data_source="abstract",
)

100%|██████████| 1/1 [00:00<00:00,  3.10it/s]


## DaG

In [18]:
dag_start = 2
dag_end = 32

# Spaced out number of sampels including total
size_of_samples = [1, 6, 11, 16, dag_end - dag_start]
number_of_samples = 50
dag_lf_range = range(dag_start, dag_end)

In [19]:
sampled_lfs_dict = {
    sample_size: (
        sample_lfs(
            list(dag_lf_range),
            len(list(dag_lf_range)),
            sample_size,
            number_of_samples,
            random_state=100,
        )
    )
    for sample_size in size_of_samples
}

In [20]:
data_columns += train_generative_label_function_sampler(
    filtered_L_abstracts,
    L_dev,
    sampled_lfs_dict,
    lf_columns_base=lf_columns_base,
    candidate_id_field=candidate_id_field,
    dev_column_base=dev_column_base,
    search_grid=grid,
    marginals_df_file=str(
        notebook_output_dir / Path("dag_predicts_ctd_training_marginals.tsv")
    ),
    curated_label="curated_ctd",
    entity_label="DaG",
    data_source="abstract",
)

100%|██████████| 1/1 [00:00<00:00,  1.07it/s]


## CtD

In [21]:
ctd_start = 32
ctd_end = 54

# Spaced out number of sampels including total
size_of_samples = [1, 6, 11, 16, ctd_end - ctd_start]
number_of_samples = 50
ctd_lf_range = range(ctd_start, ctd_end)

In [22]:
sampled_lfs_dict = {
    sample_size: (
        sample_lfs(
            list(ctd_lf_range),
            len(list(ctd_lf_range)),
            sample_size,
            number_of_samples,
            random_state=100,
        )
    )
    for sample_size in size_of_samples
}

In [23]:
data_columns += train_generative_label_function_sampler(
    filtered_L_abstracts,
    L_dev,
    sampled_lfs_dict,
    lf_columns_base=lf_columns_base,
    candidate_id_field=candidate_id_field,
    dev_column_base=dev_column_base,
    search_grid=grid,
    marginals_df_file=str(
        notebook_output_dir / Path("ctd_predicts_ctd_training_marginals.tsv")
    ),
    curated_label="curated_ctd",
    entity_label="CtD",
    data_source="abstract",
)

100%|██████████| 1/1 [00:00<00:00,  1.36it/s]


## CbG

In [24]:
cbg_start = 54
cbg_end = 74

# Spaced out number of sampels including total
size_of_samples = [1, 6, 11, 16, cbg_end - cbg_start]
number_of_samples = 50
cbg_lf_range = range(cbg_start, cbg_end)

In [25]:
sampled_lfs_dict = {
    sample_size: (
        sample_lfs(
            list(cbg_lf_range),
            len(list(cbg_lf_range)),
            sample_size,
            number_of_samples,
            random_state=100,
        )
    )
    for sample_size in size_of_samples
}

In [26]:
data_columns += train_generative_label_function_sampler(
    filtered_L_abstracts,
    L_dev,
    sampled_lfs_dict,
    lf_columns_base=lf_columns_base,
    candidate_id_field=candidate_id_field,
    dev_column_base=dev_column_base,
    search_grid=grid,
    marginals_df_file=str(
        notebook_output_dir / Path("cbg_predicts_ctd_training_marginals.tsv")
    ),
    curated_label="curated_ctd",
    entity_label="CbG",
    data_source="abstract",
)

100%|██████████| 1/1 [00:00<00:00,  1.51it/s]


## GiG

In [27]:
gig_start = 74
gig_end = 102

# Spaced out number of sampels including total
size_of_samples = [1, 6, 11, 16, gig_end - gig_start]
number_of_samples = 50
gig_lf_range = range(gig_start, gig_end)

In [28]:
sampled_lfs_dict = {
    sample_size: (
        sample_lfs(
            list(gig_lf_range),
            len(list(gig_lf_range)),
            sample_size,
            number_of_samples,
            random_state=100,
        )
    )
    for sample_size in size_of_samples
}

In [29]:
data_columns += train_generative_label_function_sampler(
    filtered_L_abstracts,
    L_dev,
    sampled_lfs_dict,
    lf_columns_base=lf_columns_base,
    candidate_id_field=candidate_id_field,
    dev_column_base=dev_column_base,
    search_grid=grid,
    marginals_df_file=str(
        notebook_output_dir / Path("gig_predicts_ctd_training_marginals.tsv")
    ),
    curated_label="curated_ctd",
    entity_label="GiG",
    data_source="abstract",
)

100%|██████████| 1/1 [00:00<00:00,  1.13it/s]


# Full Text

## DaG

In [30]:
dag_start = 2
dag_end = 32

# Spaced out number of sampels including total
size_of_samples = [1, 6, 11, 16, dag_end - dag_start]
number_of_samples = 50
dag_lf_range = range(dag_start, dag_end)

In [31]:
sampled_lfs_dict = {
    sample_size: (
        sample_lfs(
            list(dag_lf_range),
            len(list(dag_lf_range)),
            sample_size,
            number_of_samples,
            random_state=100,
        )
    )
    for sample_size in size_of_samples
}

In [32]:
data_columns += train_generative_label_function_sampler(
    L_full_text,
    L_dev,
    sampled_lfs_dict,
    lf_columns_base=lf_columns_base,
    candidate_id_field=candidate_id_field,
    dev_column_base=dev_column_base,
    search_grid=grid,
    marginals_df_file=str(
        notebook_output_dir / Path("dag_predicts_ctd_training_marginals_full_text.tsv")
    ),
    curated_label="curated_ctd",
    entity_label="DaG",
    data_source="full_text",
)

100%|██████████| 1/1 [00:01<00:00,  1.62s/it]


## CtD

In [33]:
ctd_start = 32
ctd_end = 54

# Spaced out number of sampels including total
size_of_samples = [1, 6, 11, 16, ctd_end - ctd_start]
number_of_samples = 50
ctd_lf_range = range(ctd_start, ctd_end)

In [34]:
sampled_lfs_dict = {
    sample_size: (
        sample_lfs(
            list(ctd_lf_range),
            len(list(ctd_lf_range)),
            sample_size,
            number_of_samples,
            random_state=100,
        )
    )
    for sample_size in size_of_samples
}

In [35]:
data_columns += train_generative_label_function_sampler(
    L_full_text,
    L_dev,
    sampled_lfs_dict,
    lf_columns_base=lf_columns_base,
    candidate_id_field=candidate_id_field,
    dev_column_base=dev_column_base,
    search_grid=grid,
    marginals_df_file=str(
        notebook_output_dir / Path("ctd_predicts_ctd_training_marginals_full_text.tsv")
    ),
    curated_label="curated_ctd",
    entity_label="CtD",
    data_source="full_text",
)

100%|██████████| 1/1 [00:01<00:00,  1.27s/it]


## CbG

In [36]:
cbg_start = 54
cbg_end = 74

# Spaced out number of sampels including total
size_of_samples = [1, 6, 11, 16, cbg_end - cbg_start]
number_of_samples = 50
cbg_lf_range = range(cbg_start, cbg_end)

In [37]:
sampled_lfs_dict = {
    sample_size: (
        sample_lfs(
            list(cbg_lf_range),
            len(list(cbg_lf_range)),
            sample_size,
            number_of_samples,
            random_state=100,
        )
    )
    for sample_size in size_of_samples
}

In [38]:
data_columns += train_generative_label_function_sampler(
    L_full_text,
    L_dev,
    sampled_lfs_dict,
    lf_columns_base=lf_columns_base,
    candidate_id_field=candidate_id_field,
    dev_column_base=dev_column_base,
    search_grid=grid,
    marginals_df_file=str(
        notebook_output_dir / Path("cbg_predicts_ctd_training_marginals_full_text.tsv")
    ),
    curated_label="curated_ctd",
    entity_label="CbG",
    data_source="full_text",
)

100%|██████████| 1/1 [00:01<00:00,  1.13s/it]


## GiG

In [39]:
gig_start = 74
gig_end = 102

# Spaced out number of sampels including total
size_of_samples = [1, 6, 11, 16, gig_end - gig_start]
number_of_samples = 50
gig_lf_range = range(gig_start, gig_end)

In [40]:
sampled_lfs_dict = {
    sample_size: (
        sample_lfs(
            list(gig_lf_range),
            len(list(gig_lf_range)),
            sample_size,
            number_of_samples,
            random_state=100,
        )
    )
    for sample_size in size_of_samples
}

In [41]:
data_columns += train_generative_label_function_sampler(
    L_full_text,
    L_dev,
    sampled_lfs_dict,
    lf_columns_base=lf_columns_base,
    candidate_id_field=candidate_id_field,
    dev_column_base=dev_column_base,
    search_grid=grid,
    marginals_df_file=str(
        notebook_output_dir / Path("gig_predicts_ctd_training_marginals_full_text.tsv")
    ),
    curated_label="curated_ctd",
    entity_label="GiG",
    data_source="full_text",
)

100%|██████████| 1/1 [00:01<00:00,  1.56s/it]


# Write Performance to File

In [42]:
performance_df = pd.DataFrame.from_records(data_columns)
performance_df

,lf_sample,aupr,auroc,bce_loss,sampled_lf_name,entity_label,data_source
0,2,0.232048,0.660440,0.799681,"LF_HETNET_PHARMACOTHERAPYDB,LF_HETNET_CD_ABSENT",CtD,abstract
1,1,0.214607,0.607970,0.648336,LF_DG_DISTANCE_SHORT,DaG,abstract
2,1,0.208869,0.592538,0.689486,LF_DG_GENETIC_ABNORMALITIES,DaG,abstract
3,1,0.211152,0.599061,0.682714,LF_DG_CONTEXT_SWITCH,DaG,abstract
4,1,0.221866,0.582250,0.884624,LF_DaG_DISEASE_SAMPLE,DaG,abstract
...,...,...,...,...,...,...,...
1996,28,0.235749,0.562371,1.375751,"LF_GiG_BINDING_IDENTIFICATIONS,LF_GiG_CELL_IDE...",GiG,full_text
1997,28,0.235749,0.562371,1.375751,"LF_GiG_BINDING_IDENTIFICATIONS,LF_GiG_CELL_IDE...",GiG,full_text
1998,28,0.235749,0.562371,1.375751,"LF_GiG_BINDING_IDENTIFICATIONS,LF_GiG_CELL_IDE...",GiG,full_text
1999,28,0.235749,0.562371,1.375751,"LF_GiG_BINDING_IDENTIFICATIONS,LF_GiG_CELL_IDE...",GiG,full_text


In [43]:
(
    performance_df
    >> ply.call(
        "to_csv",
        str(Path("output/performance") / Path("CtD_performance.tsv")),
        index=False,
        sep="\t",
    )
)